### Import the necessary libraries if you are starting a new notebook. Using the same data as the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv

In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
data = pd.read_csv('/Users/Hector_Martin/Documents/Labs/lab-cleaning-categorical-data/files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')
display(data.head())
print(data.shape)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


(9134, 24)


### Find all of the categorical data. Save it in a categorical_df variable. 

In [33]:
data_cat = data.select_dtypes(['object'])

display(data_cat.head())
print(data_cat.shape)

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


(9134, 16)


### Check for NaN values.

In [ ]:
data_cat.isna().sum()

### Check all unique values of columns.

In [ ]:
for i in data_cat.columns:
    print(i, data_cat[i].unique())

### Check dtypes. Do they all make sense as categorical data?

In [11]:
#Renew Offer Type could have been just a number, the prefix 'Offer' 
#does not add anything to the rows values of the column.
data_cat.dtypes

Customer             object
State                object
Response             object
Coverage             object
Education            object
Effective To Date    object
EmploymentStatus     object
Gender               object
Location Code        object
Marital Status       object
Policy Type          object
Policy               object
Renew Offer Type     object
Sales Channel        object
Vehicle Class        object
Vehicle Size         object
dtype: object

### Does any column contain alpha and numeric data? Decide how to clean it.

In [35]:
data_cat['Renew Offer Type'].unique()

array(['Offer1', 'Offer3', 'Offer2', 'Offer4'], dtype=object)

In [19]:
#Yes, Customer, Policy, Renew Offer Type

#We will drop the Customer column:

data_cat = data_cat.drop(['Customer'], axis =1)

KeyError: "['Customer'] not found in axis"

In [22]:
#for the other 2 Categoricals I will use an Ordinal Encoder because there seems to be an implicit order
#also for the sake of practicing a different encoding method:

#We are going to split the Categorical Data between Ordinal and Nominal:

data_cat_ord = data_cat[['Policy', 'Renew Offer Type']]

In [25]:
#Now the actual Ordinal Encoding:

#Policy:

mapping = {'Personal L1':0,'Personal L2':1, 'Personal L3':2,
           'Corporate L1':3,'Corporate L2':4, 'Corporate L3':5,
          'Special L1':6,'Special L2':7, 'Special L3':8}
data_cat_ord = data_cat_ord.replace({'Policy': mapping})

#Renew Offer Type:

mapping2 = {'Offer1':0,'Offer2':1, 'Offer3':2, 'Offer4':3}
data_cat_ord = data_cat_ord.replace({'Renew Offer Type': mapping2})

### Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.

In [6]:
# I would do One Hot Encoder on the rest of the Categorical columns
# I do not think that our model would benefit from creating separate columns from any alteration on Effective to Date
# so I would consider removing it.

### Compare policy_type and policy. What information is contained in these columns. 

In [36]:
#Can you identify what is important?

#Policy and Policy Type seem to show the different kinds of policies that each customer has,
#divided into 3 main categories: Personal, Corporate, Special, which implies that there must be an implicit order
#between them

#Policy seems to show more explicitly a potential implicit order, further disecting the categories
# into 3 subgroups per category.

In [29]:
data_cat['Policy Type'].unique()

array(['Corporate Auto', 'Personal Auto', 'Special Auto'], dtype=object)

### Check number of unique values in each column, can they be combined in any way to ease encoding?

In [32]:
for i in data_cat.columns:
    print(i, data_cat[i].nunique())

State 5
Response 2
Coverage 3
Education 5
Effective To Date 59
EmploymentStatus 5
Gender 2
Location Code 3
Marital Status 3
Policy Type 3
Policy 9
Renew Offer Type 4
Sales Channel 4
Vehicle Class 6
Vehicle Size 3


In [ ]:
#We could extract the days of the week on the Effective to Date column, 
#then make 2 groups between week days and weekend days if we wanted to observe how those may affect our predictions
#we could do the same with months, seasons, if it was the case, but here I do not see the point apart
#from removing this column.